In [ ]:
from datasets import load_dataset
from model import NERClassifier
from preprocess_dataset import NERDataset 
from trainner import Trainner
from transformers import BertTokenizer, BertTokenizerFast
from preprocess_dataset import remove_empty_entries

import torch

## Setups

In [ ]:
CHECKPOINT = 'neuralmind/bert-base-portuguese-cased'
tokenizer = BertTokenizerFast.from_pretrained(CHECKPOINT)

In [ ]:
data = "lener_br"
dataset = load_dataset(data)
dataset = remove_empty_entries(dataset)

In [ ]:
MAX_LEN = 256
n_labels = 13 #TODO: por enquanto hardcodado 

In [ ]:
pytorch_dataset_train = NERDataset(data=dataset['train'], max_len=MAX_LEN, tokenizer=tokenizer)
teste = pytorch_dataset_train[0]

In [ ]:
teste

## Checando se a miha correspondencia target=-100 com tokens inuteis bate

In [13]:
ids = teste['input_ids']
targets = teste['targets']

for idx, idt in enumerate(ids.squeeze()):
    print(tokenizer.convert_ids_to_tokens(idt.item()), targets[idx])

[CLS] tensor(-100)
E tensor(0)
##ME tensor(-100)
##NT tensor(-100)
##A tensor(-100)
: tensor(0)
AP tensor(0)
##EL tensor(-100)
##A tensor(-100)
##Ç tensor(-100)
##ÃO tensor(-100)
C tensor(0)
##Í tensor(-100)
##V tensor(-100)
##EL tensor(-100)
- tensor(0)
A tensor(0)
##Ç tensor(-100)
##ÃO tensor(-100)
DE tensor(0)
IN tensor(0)
##DE tensor(-100)
##N tensor(-100)
##I tensor(-100)
##Z tensor(-100)
##A tensor(-100)
##Ç tensor(-100)
##ÃO tensor(-100)
P tensor(0)
##OR tensor(-100)
D tensor(0)
##AN tensor(-100)
##OS tensor(-100)
M tensor(0)
##OR tensor(-100)
##A tensor(-100)
##IS tensor(-100)
- tensor(0)
PR tensor(0)
##EL tensor(-100)
##IM tensor(-100)
##IN tensor(-100)
##AR tensor(-100)
- tensor(0)
AR tensor(0)
##G tensor(-100)
##U tensor(-100)
##ID tensor(-100)
##A tensor(-100)
P tensor(0)
##EL tensor(-100)
##O tensor(-100)
M tensor(1)
##IN tensor(-100)
##IS tensor(-100)
##T tensor(-100)
##É tensor(-100)
##RI tensor(-100)
##O tensor(-100)
P tensor(2)
##Ú tensor(-100)
##B tensor(-100)
##L ten

***

In [ ]:
model = NERClassifier(n_labels=n_labels, checkpoint=CHECKPOINT)

## Passando dados pelo modelo

In [ ]:
input_ids = teste['input_ids']
attention_mask = teste['attention_mask']
labels = teste['targets']

In [ ]:
out = model(input_ids, attention_mask, labels=labels)

In [ ]:
print(out)

## Explorando um pouco o dataset

In [ ]:
type(dataset)

In [ ]:
dataset

In [ ]:
max_vals = [torch.max(val['targets']) for val in pytorch_dataset_train]

In [ ]:
interval = (min(max_vals).item(), max(max_vals).item()

## Testando o DataLoader

In [ ]:
from torch.utils.data import DataLoader

In [ ]:
batch_size = 8

In [ ]:
pytorch_dataset_train

In [ ]:
teste = iter(DataLoader(pytorch_dataset_train ,batch_size=batch_size))

In [ ]:
mini_batch = next(teste)

In [ ]:
mini_batch

## Passando dados pelo modelo em batch

In [ ]:
ids = mini_batch['input_ids'].squeeze().to("cuda", dtype = torch.long) # Remove a dimensão 1 ali do meio, talvez funcione?
att = mini_batch['attention_mask'].to("cuda", dtype = torch.long)
targets = mini_batch['targets'].to("cuda", dtype = torch.long)

In [ ]:
targets[0] # Labels pra uma frase: 256 tokens.

In [ ]:
targets.shape

In [ ]:
print(ids.shape)

In [ ]:
model = model.to("cuda")

In [ ]:
outs = model(ids, att, labels=targets)

In [ ]:
outs

## Convertendo predições para etiquetas

Ja vimos como converter um output de batch_size=1 para labels. Agora vamos tentar pra um batch inteiro

### TODO
- [ ] Mover a lógica de Sigmoid pra dentro do batch_to_labels

In [ ]:
from utils import batch_to_labels, preds_to_label_idx

In [ ]:
from torch.nn import Sigmoid

In [ ]:
s = Sigmoid()

In [ ]:
b_logits = outs.logits
probs = s(b_logits)

In [ ]:
probs.shape

In [ ]:
preds_to_label_idx(probs[0])

In [ ]:
outputs = batch_to_labels(b_logits)

In [ ]:
outputs = outputs.to('cuda')

In [ ]:
outputs.shape

In [ ]:
targets.shape

In [ ]:
targets.to('cuda')

In [ ]:
teste = outputs*2 # Operações no vetor original gerando um novo vetor permanecem no mesmo device

In [ ]:
teste == targets

In [ ]:
a = torch.tensor([[1,2,3], [4,5,6]])

In [ ]:
b = torch.tensor([[0,2,2], [4,5,3]])

In [ ]:
torch.sum(a == b).item()